Setup code

In [7]:
%load_ext dotenv
%dotenv
# set UPSTAGE_API_KEY in .env file
# UPSTAGE_API_KEY=your_api_key

import warnings
warnings.filterwarnings("ignore")

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [8]:
text = """
We introduce SOLAR 10.7B, a large language model (LLM) with 10.7 billion parameters, 
demonstrating superior performance in various natural language processing (NLP) tasks. 
Inspired by recent efforts to efficiently up-scale LLMs, 
we present a method for scaling LLMs called depth up-scaling (DUS), 
which encompasses depthwise scaling and continued pretraining.
In contrast to other LLM up-scaling methods that use mixture-of-experts, 
DUS does not require complex changes to train and inference efficiently. 
We show experimentally that DUS is simple yet effective 
in scaling up high-performance LLMs from small ones. 
Building on the DUS model, we additionally present SOLAR 10.7B-Instruct, 
a variant fine-tuned for instruction-following capabilities, 
surpassing Mixtral-8x7B-Instruct. 
SOLAR 10.7B is publicly available under the Apache 2.0 license, 
promoting broad access and application in the LLM field.
"""

query = f"""
Make 3 questions using this text.

{text}
"""

query2 = """
Linear transformation is a function that satisfies two features.
One is homogenity, and other is additivity.

Linear transformation of the subspace is also subspace.
Prove or disprove this statement based on given contents.
"""

query3 = """
The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?
"""

query4 = """
What is a Mahjong game?
"""

In [25]:
from langchain_upstage import ChatUpstage
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatUpstage()

prompt_template = PromptTemplate.from_template(
    """
In this query, separate the instruction and context.
Do not answer to the query, just separate it. Also, only use given query, do not generate new words.
Context is can be a base information or some text to convert or manipulate.
Context can be not provided(N/A for this case).
Instruction and context must not have duplicate contents.
Print instruction as first paragraph and context as second paragraph.
---
Query : {query}
"""
)
chain = prompt_template | llm | StrOutputParser()
answer = chain.invoke({"query": query})

instr, context = [s.strip() for s in answer.split("Instruction:")[1].split("Context:")]
print(instr)
print(context)

Make 3 questions using this text.
We introduce SOLAR 10.7B, a large language model (LLM) with 10.7 billion parameters, demonstrating superior performance in various natural language processing (NLP) tasks. Inspired by recent efforts to efficiently up-scale LLMs, we present a method for scaling LLMs called depth up-scaling (DUS), which encompasses depthwise scaling and continued pretraining. In contrast to other LLM up-scaling methods that use mixture-of-experts, DUS does not require complex changes to train and inference efficiently. We show experimentally that DUS is simple yet effective in scaling up high-performance LLMs from small ones. Building on the DUS model, we additionally present SOLAR 10.7B-Instruct, a variant fine-tuned for instruction-following capabilities, surpassing Mixtral-8x7B-Instruct. SOLAR 10.7B is publicly available under the Apache 2.0 license, promoting broad access and application in the LLM field.


In [27]:
# 각 chain을 실행하고 결과 누적으로 얻음
# (전체 히스토리 누적 후) 최종 결과 질문

prompt_template = PromptTemplate.from_template(
    """
For instruction "{instr}", give me what to do to achieve this instruction for AI.
Only print 1., 2., 3., ... steps to do. Do not answer to the question, and only print steps.
"""
)

chain = prompt_template | llm | StrOutputParser()
answer = chain.invoke({"instr": instr, "context": context})
print(answer)

1. Read the text carefully to understand its content.
2. Identify the main ideas or key points in the text.
3. Formulate questions that relate to these main ideas or key points.
4. Ensure that each question is clear and concise.
5. Review the questions to make sure they are relevant and interesting.
